# Intro til språkmodeller

### Spørsmål
- Vi trenger at dte går fortere å kalle på modellen :)
- Hvordan gjør vi det med nøkler? Per nå leser denne fortsatt fra .env-filen da jeg ikke ville pushe de til github.


### Tanker

- Gi mer info i prompt om situasjonen rundt spørreundersøkelsen og hva det har blitt spurt om. Spesielt i situasjonen hvor vi øsnker å oppsummere den generelle viben av feedbacken i hver kategori. Var folk fornøyde? Ønsker de tiltak for forbedring?

# AzureChatOpenAI
### Språkmodellene

In [2]:
from langchain_openai import AzureChatOpenAI
from dotenv import find_dotenv, load_dotenv
import os


# Get environment variables
load_dotenv(find_dotenv(), override=True)


# Variabler heller enn klasser
class Llm:
    """
    Class containing the language model.
    """

    llm = AzureChatOpenAI(
        azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_GPT_4O_MINI"],
        model=os.environ.get("OPENAI_MODEL_GPT_4O-MINI", default="gpt-4o-mini"),
        temperature=0,
    )


class LlmRes:
    """
    Class containing the language model.
    """

    llm = AzureChatOpenAI(
        azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_GPT_3O_MINI"],
        model=os.environ.get("OPENAI_MODEL_GPT_3O-MINI", default="o3-mini"),
        reasoning_effort="high",
    )




# Den enkleste måten å bruke AzureChatOpenAI

In [3]:
prompt = 'Hei!'
response = Llm.llm.invoke(prompt)

print(response)

print(response.content)

content='Hei! Hvordan kan jeg hjelpe deg i dag?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protect

# Structured output
### Pydantic-objekter
- Gå igjennom hva disse objektene er, hva kan du kreve av outputen?

In [5]:
# Enkel bruk av structured output 
# Hva i alle dager er pydantic? :D
# Lag en enkel klasse og kjør
from pydantic import BaseModel, Field
from typing import Literal, Optional

class Categorize(BaseModel):
    "Categorization of feedback on IT-services."

    # Thoughts: Forklar tankegangen din.
    category : str = Field(description="The best fitting general category. Only one.")
   
    

In [6]:
structured_llm = Llm.llm.with_structured_output(   #Gå igjennom denne, hvordan bruker man structured output?
        Categorize, method="function_calling"
    )

response = structured_llm.invoke("Send inn prompt + data")


In [7]:
class CategorizeFromOptions(BaseModel):
    """Categorization of IT-questionaire feedback."""

    category: Literal[
        "Network",
        "IT Training",
        "IT Support",
        'Other'
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )

In [ ]:
# Chain of thought

In [ ]:
# Find potential new categories based on feedback categorized as others

In [8]:
class CategorizeFromOptions2(BaseModel):
    """Categorization of IT-questionaire feedback."""

    category: Literal[
       # Insert the categories you found while exploring the feedback initially categorized as 'Other'.
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )
    reason : str = Field(description="Why do you believe this category is the most fitting one? Explain.")

SyntaxError: invalid syntax (3508461298.py, line 6)

### Oppgave??

In [9]:
# Kan de mekke på disse selv? Hvordan kan vi gjøre dette steget mer interaktivt


class Categorize_search(BaseModel):
    """Categorization of IT-questionaire feedback."""

    categories: Literal[
        "Network",
        "IT Training",
        "IT Support",
        'Other'
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )
    #if_other: str = Field(
        #description="If you chose to categorize the feedback as 'Other', return an explanation as to why and what category you think would be the best fit for the feedback."
    #)


class Categorize_bound(BaseModel):
    # Begrenser kategoriene modellen kan velge mellom. Den får kun lov til å putte feedbacken i en av de forhåndsbestemte kategoriene.
    """Categorization of IT-questionaire feedback."""

    categories: Literal[
        "Cyber security",
        "IT training",
        "IT support",
        "Quality of technology",
        "Data quality",
        "Network",
    ] = Field(description="Categorize the feedback into the most fitting category.")
   # rating: Optional[int] = Field(
       # description="Your certainty of the corectness of the best category on a scale from 1-10"
  #  )
    #reason: str = Field(
    #    description="Give a short scentence as to why you think this category is the best fit."
   # )



class Categorize(BaseModel):
    "Categorization of feedback on IT-services."

    # Thoughts: Forklar tankegangen din.
    cat_1: str = Field(description="The best fitting general category. Only one.")
    
class Summarize(BaseModel):

    # 1. Oppsummer kategori, tar inn alle feedbacks under 1 kategori
    # 2. Likely underlying causes of recurring issues or praises



_IncompleteInputError: incomplete input (2397846824.py, line 51)

# Case

# Datasettet (FLYTT TIL INTRO)

In [10]:
import pandas as pd


class FeedbackData:
    """
    Class containing fake datasets by us :)
    """

    # Load a DataFrame with a specific version of a CSV
    file_path = "../files/raw_data_LLM101.csv"
    df_o = pd.read_csv(file_path)

    enr_path = "../files/random_out.csv"
    df = pd.read_csv(enr_path)#.head(20)
    df_mini = df.head(5)


# print(FeedbackData.df["Feedback"][0])


# Kategorisering

In [ ]:
# Lag en funksjon som tar inn enten full prompt eller data fra undersøkelsen + pydantic objekt (struct) + modell og returnerer output fra modell

In [12]:
## KURS
def categorize_feedback(feedback_txt : str, struktur, llm_model) -> dict:
    # Prompt
    task = f"""
    Catgorize the feedback: {feedback_txt}.
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(   #Gå igjennom denne, hvordan bruker man structured output?
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

In [13]:
def categorize_feedback(feedback_txt : str, struktur, llm_model) -> dict:
    # Prompt
    task = f"""
    The following feedback is from an internal survey at 'IT and Things Company' where they asked their employees for feedback on their IT-services in general.
    Catgorize the feedback: {feedback_txt}.
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

# Din tur

In [ ]:
# Returner en dataframe med kategoriserte feedback 
# 1. Be den kategorisere uten begrensninger
# 2. Gi den et sett med forhåndsdefinerte kategorier
# 3. La 'other' være et alternativ. 

In [16]:
## returnerer kategorisert feedback

responses = []

for f in FeedbackData.df["Feedback"]:
    feedback_4O = categorize_feedback(feedback_txt = f, struktur = CategorizeFromOptions, llm_model = Llm.llm)
    feedback_3O = categorize_feedback(feedback_txt = f, struktur = CategorizeFromOptions, llm_model = LlmRes.llm)
    responses.append({'Feedback': f, 'Category_4O': feedback_4O['category'], 'Category_3O': feedback_3O['category']})
    #print(f)

    
cat_df = pd.DataFrame(responses)
print(cat_df)

                                             Feedback  Category_4O  \
0   The recent training module on cybersecurity pr...  IT Training   
1   I appreciate the updated training materials fo...  IT Training   
2   The scheduled training sessions have been cons...  IT Training   
3   In the recent workshop for our CRM system, the...  IT Training   
4   Training on modern data analytics has become m...  IT Training   
5   Our hands-on training for remote work tools ha...  IT Training   
6   The new onboarding process for IT training rem...  IT Training   
7   I found the team training on system backups bo...  IT Training   
8   The recurring training updates about IT securi...  IT Training   
9   Our digital transformation training is well-st...  IT Training   
10  Our network performance has been erratic, with...      Network   
11  The daily hiccups in our VPN and internal conn...      Network   
12  Network instability during peak hours leaves m...      Network   
13  Although the net

In [ ]:
# Identifiser de som ble identifisert som 'other' og finn kategori for de. 

In [ ]:
others_df = cat_df[(cat_df['Category'] == 'Other') | (cat_df['Category2'] == 'Other')]
print(others_df)

In [ ]:
## returnerer kategorisert feedback

# Heller send inn alle others-feedback samtidig og be den finne X nye kategorier. 
responses_new_cat = []

for f in others_df["Feedback"]:
    output_i = categorize_feedback(feedback_txt = f, struktur = Categorize, llm_model = Llm.llm)
    responses_new_cat.append({'Feedback': f,'Category': output_i["cat_1"]})

new_cat_df = pd.DataFrame(responses_new_cat)
print(new_cat_df)

In [ ]:
# Ny kategorisering hvor vi gir den grunnkategorier + de modellen har gjenkjent under other 

# Reasoning models - when we want the LLM to return logic analyzation across rows

Nå vil vi teste resoneringsmodellen, sammenlikne?

## Summarization per category

- Få modellen til å oppsummere per kategori.

In [ ]:
# Now that we have defined fitting categories for the feedback, we can use a reasoning model to create a summary of the rows per category to provide an overall overview
def summarize(feedback_txt : str, struktur, llm_model) -> dict:

    # Prompt
    task = f"""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {feedback_txt}
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response_sum = structured_llm.invoke(task)
    # Return the response
    return response_sum.__dict__

# Print head of response


## Generating a report for the leadership

- Bruk oppsummeringen til å generere en en rapport til ledelsen med forlag til forbedringspotensiale i IT. Hvilke tiltak bør bedriften gjøre?

In [ ]:
def report(feedback_txt : str, struktur, llm_model) -> dict:

    # Prompt
    task = f"""
You are an expert HR and technical operations analyst. I will provide you with a dataset of employee feedback collected from an IT company.

Your task is to deeply analyze this feedback and generate a concise executive-level summary report in markdown format that includes:

1. Key Takeaways
Provide a short summary of the overall feedback in 3-5 bullet points. Focus only on the main issues or areas of satisfaction.
Include both positive and negative themes, but prioritize the most important and impactful points.
Limit each point to 1-2 sentences.
Before finalizing each point, take a moment to reflect on why each issue might be present (e.g., systemic problems, temporary issues, resource constraints, etc.)

2. Suggested Improvements
Based on the overall feedback, propose 2-3 high-level, actionable measures that the company could take to address the most pressing issues and enhance overall performance or satisfaction.
Each suggestion should be brief, directly tied to the feedback, and strategic in nature.
Think about short-term vs long-term solutions and consider the feasibility of each suggestion.

3. Output
Present your findings in a structured way with clear section headings, bullet points for easy scanning, and a consise, direct and professional tone suitable for leadership review.

This is the employee feedback data: {feedback_txt}
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

1. Skulle man hatt en oppgave til slutt for de som er fort ferdig hvor de sender inn hele undersøkelsen og ber modellen velge beste kategorier før vi igjen bruker de kategoriene i structured output for å kategorisere. kjør så hele på nytt og se om du synes resultatene ble bedre. 

2. Hvordan kunne dette vært gjort bedre? Er det 